In [0]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import numpy as np
import pandas as pd
%matplotlib inline

Order_UpTo_Level(SKU, ТТ) - политика пополнения (S)
<br>ReOrder_Level(SKU, ТТ) - политика пополнения (s)
<br>(Возможное представление — Dataframe)

In [0]:
OUL = 1 # Order Upto Level
ROL = 0 # Re-Order Level
Delta = OUL - ROL # Delta = Order_UpTo_Level - ReOrder_Level (S-s)

# Восстановление спроса

Используем обучающие данные из файла data.zip, чтобы научиться восстанавливать спрос.

In [0]:
sales = pd.read_csv('/content/gdrive/My Drive/A Reinforcement Learning Approach for Inventory Optimization in Retail/DemandRestoring_v0/data/sales_4001.csv', sep=',', decimal='.')
sales['Timestamp'] = pd.to_datetime(sales[u'purchase_date'])
sales = sales.set_index(['Timestamp'], drop=True).rename(columns={'quantity':'sales'})
sales.head()

,purchase_date,shop_id,product_id,price,sales,price_after_discount
Timestamp,,,,,,
2013-01-02,2013-01-02,4001,4100024369,1480,1,1480.0
2013-01-02,2013-01-02,4001,4100024794,2807,1,2807.0
2013-01-02,2013-01-02,4001,4100028657,979,1,979.0
2013-01-02,2013-01-02,4001,4300000205,3319,1,3319.0
2013-01-02,2013-01-02,4001,4300007581,1899,1,1899.0


In [0]:
inventory = pd.read_csv('/content/gdrive/My Drive/A Reinforcement Learning Approach for Inventory Optimization in Retail/DemandRestoring_v0/data/remains_4001.csv', sep=',', decimal='.')
inventory['Timestamp'] = pd.to_datetime(inventory[u'updated_at'])
inventory = inventory.set_index(['Timestamp'], drop=True).rename(columns={'quantity':'inventory'})
# inventory = inventory.sort_index()
inventory.head()

,shop_id,product_id,updated_at,inventory
Timestamp,,,,
2015-08-31,4001,4300059161,2015-08-31,0
2016-01-02,4001,4300059161,2016-01-02,0
2015-08-31,4001,4300059161,2015-08-31,1
2013-04-17,4001,4300051009,2013-04-17,1
2013-08-11,4001,4300051009,2013-08-11,0


В качестве списка всех магазинов и товаров используем информацию об остатках, так как при наличии записи о ненулевом остатке в inventory можно говорить об нулевых продажах в sales.

In [0]:
shops = np.sort(inventory['shop_id'].unique())
print(len(shops), shops)

products = np.sort(inventory['product_id'].unique())
print(len(products), products)

1 [4001]
14466 [4100000013 4100000017 4100000029 ... 4900005059 4900005473 4900005475]


Для каждого товара в каждом магазине расчитаем lambda.



In [0]:
lambda_data = pd.DataFrame(columns=['shop_id', 'product_id', 'month', 'lambda'])
lambda_data

,shop_id,product_id,month,lambda


In [0]:
shop_id = 4001
product_id = 4100000013

# inventory = inventory[(inventory['product_id'] == product_id) & (inventory['shop_id'] == shop_id)]
# inventory[(inventory['product_id'] == product_id) & (inventory['shop_id'] == shop_id)]

# Агрегируем на всякий случай остатки на одну дату (берём максимально известный остаток за дату)
iv_ts = inventory.reset_index().groupby('Timestamp').agg({'inventory':np.max})

# Добавим пропущенные даты и заполним значения остатков в эти даты нулями
iv_ts = iv_ts.reindex(pd.date_range(np.min(iv_ts.index), np.max(iv_ts.index))).fillna(method='ffill')

# Добавим данные о продажах
sales_ts = sales[(sales['product_id'] == product_id) & (sales['shop_id'] == shop_id)]

# Объединим продажи и остатки в один датафрейм
iv_sales = sales_ts[['sales']].merge(iv_ts, how='right', left_index=True, right_index=True)
iv_sales['sales'].fillna(0, inplace=True)

# Удалим начало и "хвост" истории, когда продажи и остатки были равны нулю
positive_iv_sales = iv_sales[iv_sales.max(axis=1) > 0]
life_start_date = positive_iv_sales.index[0]
life_end_date = positive_iv_sales.index[-1]
iv_sales = iv_sales[(iv_sales.index >= life_start_date) & (iv_sales.index <= life_end_date)] 

# Дни, когда и остатки и продажи = нулю
zero_idx = (iv_sales['inventory'] == 0) & (iv_sales['sales'] == 0)
# Дни, когда продажи = остаткам
sales_equal_inv_idx = (iv_sales['inventory'] == iv_sales['sales']) 
# Дни, когда продажи >= остаткам
sales_greater_i_idx = (iv_sales['inventory'] <= iv_sales['sales']) 

# Суммарные продажи товара в данном магазине
sum_k = iv_sales['sales'].sum()
# Количество дней, когда продажи были меньше остатка
n_k_less_m = iv_sales['sales'][(~zero_idx) & (~sales_greater_i_idx)].count()
# Количество дней, когда продажи были равны остатку (все распродано)
n_k_equal_m = iv_sales['sales'][(~zero_idx) & sales_greater_i_idx].count()

alpha = 1
LAMBDA = sum_k / (n_k_less_m + alpha * n_k_equal_m)

print('sum_k: ', sum_k)
print('n_k_less_m: ', n_k_less_m)
print('n_k_equal_m: ', n_k_equal_m)
print('LAMBDA:', LAMBDA)

sum_k:  1.0
n_k_less_m:  179
n_k_equal_m:  1
LAMBDA: 0.005555555555555556


In [0]:
def DemandRestoring(shop_id, product_id):
  # inventory = inventory[(inventory['product_id'] == product_id) & (inventory['shop_id'] == shop_id)]
  # inventory[(inventory['product_id'] == product_id) & (inventory['shop_id'] == shop_id)]

  # Агрегируем на всякий случай остатки на одну дату (берём максимально известный остаток за дату)
  iv_ts = inventory.reset_index().groupby('Timestamp').agg({'inventory':np.max})

  # Добавим пропущенные даты и заполним значения остатков в эти даты нулями
  iv_ts = iv_ts.reindex(pd.date_range(np.min(iv_ts.index), np.max(iv_ts.index))).fillna(method='ffill')

  # Добавим данные о продажах
  sales_ts = sales[(sales['product_id'] == product_id) & (sales['shop_id'] == shop_id)]

  # Объединим продажи и остатки в один датафрейм
  iv_sales = sales_ts[['sales']].merge(iv_ts, how='right', left_index=True, right_index=True)
  iv_sales['sales'].fillna(0, inplace=True)

  # Удалим начало и "хвост" истории, когда продажи и остатки были равны нулю
  positive_iv_sales = iv_sales[iv_sales.max(axis=1) > 0]
  life_start_date = positive_iv_sales.index[0]
  life_end_date = positive_iv_sales.index[-1]
  iv_sales = iv_sales[(iv_sales.index >= life_start_date) & (iv_sales.index <= life_end_date)] 

  # Дни, когда и остатки и продажи = нулю
  zero_idx = (iv_sales['inventory'] == 0) & (iv_sales['sales'] == 0)
  # Дни, когда продажи = остаткам
  sales_equal_inv_idx = (iv_sales['inventory'] == iv_sales['sales']) 
  # Дни, когда продажи >= остаткам
  sales_greater_i_idx = (iv_sales['inventory'] <= iv_sales['sales']) 

  # Суммарные продажи товара в данном магазине
  sum_k = iv_sales['sales'].sum()
  # Количество дней, когда продажи были меньше остатка
  n_k_less_m = iv_sales['sales'][(~zero_idx) & (~sales_greater_i_idx)].count()
  # Количество дней, когда продажи были равны остатку (все распродано)
  n_k_equal_m = iv_sales['sales'][(~zero_idx) & sales_greater_i_idx].count()

  alpha = 1 # Возможно проапгрейдить
  LAMBDA = sum_k / (n_k_less_m + alpha * n_k_equal_m)

  return LAMBDA

In [0]:
for shop_id in shops:
  for product_id in products:
    LAMBDA = DemandRestoring(shop_id, product_id)
    lambda_data = lambda_data.append({'shop_id': shop_id, 'product_id': product_id, 'month': 1, 'lambda': LAMBDA}, ignore_index=True)

In [0]:
# LAMBDA = DemandRestoring(shop_id, product_id)
# lambda_data = lambda_data.append({'shop_id': shop_id, 'product_id': product_id, 'month': 1, 'lambda': LAMBDA}, ignore_index=True)
# lambda_data

In [0]:
lambda_data[['shop_id', 'product_id', 'month']] = lambda_data[['shop_id', 'product_id', 'month']].astype('int64')
lambda_data['lambda'] = lambda_data['lambda'].astype('float64')
lambda_data

In [0]:
export_lambda_csv = lambda_data.to_csv('/content/gdrive/My Drive/A Reinforcement Learning Approach for Inventory Optimization in Retail/DemandRestoring_v0/data/lambda_data.csv', sep=',', decimal='.')

Можно сразу подгрузить посчитанную lambda.

In [0]:
lambda_data = pd.read_csv('/content/gdrive/My Drive/A Reinforcement Learning Approach for Inventory Optimization in Retail/DemandRestoring_v0/data/lambda_data.csv', sep=',', decimal='.')

# Остальные составляющие среды

Order(SKU, ТТ)_t - это количество товаров,отправка которого инициировано соответствующим Recommended_Order(SKU, ТТ)_t, которое финально доедет и будет выставлено на полку в магазин.

Order(SKU, ТТ)_t и Recommended_Order_t в этих обозначениях _t - дата, когда сформировались данные объёмы.

State: (Stock(SKU, ТТ)_j, Sales(SKU, ТТ)_j, Demand(SKU, ТТ)_j, Orders_Pipeline(SKU, ТТ): [Order_t, Ordrer_t-n, Order_t-m, Order_t-p], n<m<p) period t1-t2

Хранить Stock, Sales и Demand как dataframe?

LT = 1

Важно: в State продаж (Sales) и остатки (Stock) - модельные (нужно моделировать. Те, Sales и Stock, что пришли в данных от заказчика (как результат работы реальной системы) - нужны только для восстановления спрос

In [0]:
sales_data = pd.DataFrame(columns=['purchase_date',	'shop_id', 'product_id', 'price',	'sales'])
stock_data = pd.DataFrame(columns=['shop_id', 'product_id', 'updated_at',	'inventory'])

Рекомендация к заказу: Recommend_Order_t = OUL - (INV + Orders_t - 1), IF Inv_t <= ROL 0, otherwise.

In [0]:
rec_order = [0]
order = [0]
inv = [0, 0]

for t in range(1, 2):
  if inv[t] <= ROL:
    rec_order.append(OUL - inv[t] - order[t - 1])
  else:
    rec_order.append(0)

print(rec_order)

recommend_order - наша рекомендация к заказу
<br>order - объем получаемого заказа

order = F(recommend_order_t, alpha_order)
<br>1)order <= recommend_order_t 
<br>2)$\sum_{sku}$ E(order_{sku,t}) = alpha_order * E(recommend_order_{sku,t})

inv[t] = max(inv[t - 1] + order_come[t] - demand(m), 0)
<br>sales = max(demand(m), inv[t - 1] + order_come[t])

In [0]:
from math import ceil
from scipy.stats import chi

rec_order = 10
alpha_order = 0.3

order = np.random.uniform(rec_order * alpha_order, rec_order, 5)
print(order)

# Красивый вариант рассчета с использованием датафреймов (in progress)

0 bold text Объединим bold text два датафрейма sales и inventory в один (причем будем объединять только по ненулевым параметрам). 

In [0]:
# data = pd.merge(left=sales, right=inventory, how='right', left_on=['shop_id',	'product_id', 'purchase_date'], right_on=['shop_id', 'product_id', 'updated_at'])
# data = pd.merge(left=sales, right=inventory, how='right', left_index=True, right_index=True)
data = pd.merge(left=sales, right=inventory, how='outer')
data.head()

,purchase_date,shop_id,product_id,sales,updated_at,inventory
0,2013-01-02,4001,4100024369,1.0,2014-12-24,0.0
1,2014-12-24,4001,4100024369,1.0,2014-12-24,0.0
2,2013-01-02,4001,4100024794,1.0,2015-08-31,1.0
3,2013-01-02,4001,4100024794,1.0,2015-08-31,0.0
4,2013-01-02,4001,4100028657,1.0,2015-08-31,1.0


In [0]:
 data = sales[['sales', 'purchase_date',	'shop_id',	'product_id']].merge(inventory, how = 'right', left_index = True, right_index = True)
 data.head()

,sales,purchase_date,shop_id_x,product_id_x,shop_id_y,product_id_y,updated_at,inventory
Timestamp,,,,,,,,
2013-01-02,1.0,2013-01-02,4001.0,4.100024e+09,4001,4300000205,2013-01-02,0
2013-01-02,1.0,2013-01-02,4001.0,4.100025e+09,4001,4300000205,2013-01-02,0
2013-01-02,1.0,2013-01-02,4001.0,4.100029e+09,4001,4300000205,2013-01-02,0
2013-01-02,1.0,2013-01-02,4001.0,4.300000e+09,4001,4300000205,2013-01-02,0
2013-01-02,1.0,2013-01-02,4001.0,4.300008e+09,4001,4300000205,2013-01-02,0


In [0]:
sales

,purchase_date,shop_id,product_id,sales
Timestamp,,,,
2013-01-02,2013-01-02,4001,4100024369,1
2013-01-02,2013-01-02,4001,4100024794,1
2013-01-02,2013-01-02,4001,4100028657,1
2013-01-02,2013-01-02,4001,4300000205,1
2013-01-02,2013-01-02,4001,4300007581,1
...,...,...,...,...
2016-10-20,2016-10-20,4001,4300058441,1
2016-10-20,2016-10-20,4001,4300058538,1
2016-10-20,2016-10-20,4001,4300060869,1


In [0]:
data

,sales,purchase_date,shop_id_x,product_id_x,shop_id_y,product_id_y,updated_at,inventory
Timestamp,,,,,,,,
2013-01-02,1.0,2013-01-02,4001.0,4.100024e+09,4001,4300000205,2013-01-02,0
2013-01-02,1.0,2013-01-02,4001.0,4.100025e+09,4001,4300000205,2013-01-02,0
2013-01-02,1.0,2013-01-02,4001.0,4.100029e+09,4001,4300000205,2013-01-02,0
2013-01-02,1.0,2013-01-02,4001.0,4.300000e+09,4001,4300000205,2013-01-02,0
2013-01-02,1.0,2013-01-02,4001.0,4.300008e+09,4001,4300000205,2013-01-02,0
...,...,...,...,...,...,...,...,...
2016-10-20,1.0,2016-10-20,4001.0,4.300058e+09,4001,4100058973,2016-10-20,0
2016-10-20,1.0,2016-10-20,4001.0,4.300059e+09,4001,4100058973,2016-10-20,0
2016-10-20,1.0,2016-10-20,4001.0,4.300061e+09,4001,4100058973,2016-10-20,0


In [0]:
data[data['shop_id'] == 4001][data['product_id'] == 4900001067]

In [0]:
data['Timestamp'] = pd.to_datetime(data[u'updated_at'])
data = data.set_index(['Timestamp']).drop(columns=['purchase_date', 'updated_at'])
data.head()

,shop_id,product_id,sales,inventory
Timestamp,,,,
2013-01-02,4001,4300000205,1,0
2013-01-02,4001,4300007581,1,0
2013-01-03,4001,4100028568,1,0
2013-01-03,4001,4100039578,1,0
2013-01-03,4001,4300006873,1,0


In [0]:
shops = data['shop_id'].unique()
print(len(shops), shops)

1 [4001]


In [0]:
products = data['product_id'].unique()
print(len(products), products)

8114 [4300000205 4300007581 4100028568 ... 4300058538 4300060869 4300073829]


Создадим датафрейм lambda_data для хранения значения параметра lambda[shop_id][product_id][month]. 

---



In [0]:
lambda_data = pd.DataFrame(data.reset_index(), columns=['shop_id', 'product_id', 'month', 'lambda'])
lambda_data = lambda_data.sort_values(['shop_id', 'product_id']).reset_index(drop=True)
lambda_data

,shop_id,product_id,month,lambda
0,4001,4100000017,NaN,NaN
1,4001,4100000137,NaN,NaN
2,4001,4100000170,NaN,NaN
3,4001,4100000174,NaN,NaN
4,4001,4100000175,NaN,NaN
...,...,...,...,...
8496,4001,4900002021,NaN,NaN
8497,4001,4900002021,NaN,NaN
8498,4001,4900002085,NaN,NaN
8499,4001,4900002085,NaN,NaN


In [0]:
# lambda_data = lambda_data.append({'shop_id': 4002, 'product_id': 1, 'month': 1, 'lambda': 1}, ignore_index=True)
# lambda_data = lambda_data.append({'shop_id': 4002, 'product_id': 1, 'month': 1, 'lambda': 1}, ignore_index=True)
# lambda_data = lambda_data.append({'shop_id': 4002, 'product_id': 2, 'month': 1, 'lambda': 1}, ignore_index=True)
# lambda_data = lambda_data.append({'shop_id': 4002, 'product_id': 2, 'month': 1, 'lambda': 1}, ignore_index=True)
# lambda_data

In [0]:
lambda_data.drop_duplicates(subset=['shop_id', 'product_id'], keep='first', inplace=True)
lambda_data = lambda_data.reset_index(drop=True)
lambda_data

,shop_id,product_id,month,lambda
0,4001,4100000017,NaN,NaN
1,4001,4100000137,NaN,NaN
2,4001,4100000170,NaN,NaN
3,4001,4100000174,NaN,NaN
4,4001,4100000175,NaN,NaN
...,...,...,...,...
8109,4001,4900001995,NaN,NaN
8110,4001,4900002005,NaN,NaN
8111,4001,4900002021,NaN,NaN
8112,4001,4900002085,NaN,NaN


Восстановление спроса.

In [0]:
lambda_tmp = pd.DataFrame(columns=['shop_id', 'product_id', 'month', 'lambda'])
lambda_tmp

,shop_id,product_id,month,lambda


In [0]:
data

,shop_id,product_id,sales,inventory
Timestamp,,,,
2013-01-02,4001,4300000205,1,0
2013-01-02,4001,4300007581,1,0
2013-01-03,4001,4100028568,1,0
2013-01-03,4001,4100039578,1,0
2013-01-03,4001,4300006873,1,0
...,...,...,...,...
2016-10-20,4001,4100063151,1,0
2016-10-20,4001,4300058089,1,0
2016-10-20,4001,4300058538,1,0


In [0]:
data[data['shop_id'] == 4001][data['product_id'] == 4900001067]

,shop_id,product_id,sales,inventory
Timestamp,,,,
2015-09-03,4001,4900001067,1,0
2016-06-05,4001,4900001067,1,0


In [0]:
sales[sales['product_id'] == 4900001067]

,purchase_date,shop_id,product_id,sales
Timestamp,,,,
2013-01-15,2013-01-15,4001,4900001067,5
2013-01-15,2013-01-15,4001,4900001067,1
2013-01-15,2013-01-15,4001,4900001067,1
2013-01-16,2013-01-16,4001,4900001067,1
2013-01-17,2013-01-17,4001,4900001067,1
...,...,...,...,...
2016-09-06,2016-09-06,4001,4900001067,1
2016-09-12,2016-09-12,4001,4900001067,1
2016-09-13,2016-09-13,4001,4900001067,1


In [0]:
inventory[inventory['product_id'] == 4900001067]

,shop_id,product_id,updated_at,inventory
Timestamp,,,,
2015-08-31,4001,4900001067,2015-08-31,33
2015-09-03,4001,4900001067,2015-09-03,0
2015-12-12,4001,4900001067,2015-12-12,5
2016-06-05,4001,4900001067,2016-06-05,0


In [0]:
data.groupby(['product_id']).unique.count()

AttributeError: ignored

In [0]:
data.groupby(['product_id']).product_id.count()

product_id
4100000017    1
4100000137    1
4100000170    1
4100000174    1
4100000175    1
             ..
4900001995    2
4900002005    1
4900002021    3
4900002085    2
4900005475    1
Name: product_id, Length: 8114, dtype: int64